# Continuous Control of Reacher

---

This notebook is the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Packages needed

* [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) 
* [NumPy](http://www.numpy.org/)
* [pytorch](https://pytorch.org/)
* [plotly](https://plot.ly/)

In [1]:
import time
from collections import deque

import numpy as np
import torch

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import pandas as pd

init_notebook_mode(connected=True)


from unityagents import UnityEnvironment

env = UnityEnvironment(file_name='Reacher.app', no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### Train Agent: Script

In [4]:
from ddpg_agent import Agent
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)

Actor(
  (bn0): BatchNorm1d(33, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=33, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)
Critic(
  (bn0): BatchNorm1d(33, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fcs1): Linear(in_features=33, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=132, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)


In [5]:
scores = []                                                # global score

def train_ddpg(n_episodes=1000, print_every=50, max_t=10000):
       
    scores_deque = deque(maxlen=100)                       # score's window of last 100 scores
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        agent.reset()                                      # reset the agent
        state = env_info.vector_observations[0]            # get the current state
        score = 0                                          # reset score for an episod
        
        for t in range(max_t):
            action = agent.act(state)                      # select an action       
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(
                state, action, reward, next_state, done)   # take an action
            score += reward                                # update the score of the expisod
            state = next_state                             # roll over the state to next time step
            if done:                                       # exit loop if episode finished
                break
        
        scores_deque.append(score)                         # save score to the window
        scores.append(score)                               # save scores to the global list

        text =\
            '\rEpisode {} \tLast 100 Episodes Score Average, Min, Max: {:.3f}, {:.2f}, {:.2f} \tLast Score: {:.2f}'\
            .format(i_episode, np.mean(scores_deque), np.min(scores_deque), np.max(scores_deque), score)

        if (i_episode % print_every == 0) or (np.mean(scores_deque) > 30.0):
            print(text)
            agent.save_checkpoints()
        else:
            print(text, end="")
        
        if np.mean(scores_deque) > 30.0:
            print('Done :)')
            break


### Train the agent

In [6]:
scores = []
train_ddpg(n_episodes = 1000, print_every=25)

Episode 25 	Last 100 Episodes Score Average, Min, Max: 1.217, 0.13, 5.85 	Last Score: 1.61
Episode 50 	Last 100 Episodes Score Average, Min, Max: 1.963, 0.13, 5.85 	Last Score: 2.93
Episode 75 	Last 100 Episodes Score Average, Min, Max: 3.211, 0.13, 14.94 	Last Score: 3.084
Episode 100 	Last 100 Episodes Score Average, Min, Max: 4.773, 0.13, 25.84 	Last Score: 15.04
Episode 125 	Last 100 Episodes Score Average, Min, Max: 8.236, 0.91, 29.16 	Last Score: 15.29
Episode 150 	Last 100 Episodes Score Average, Min, Max: 13.892, 2.36, 35.98 	Last Score: 31.21
Episode 175 	Last 100 Episodes Score Average, Min, Max: 19.587, 3.89, 39.56 	Last Score: 26.14
Episode 200 	Last 100 Episodes Score Average, Min, Max: 24.513, 2.89, 39.56 	Last Score: 37.45
Episode 225 	Last 100 Episodes Score Average, Min, Max: 29.180, 2.89, 39.56 	Last Score: 34.90
Episode 242 	Last 100 Episodes Score Average, Min, Max: 30.030, 2.89, 39.56 	Last Score: 30.56
Done :)


In [7]:
data = [go.Scatter(x=np.arange(len(scores)), y=scores)]
layout = dict(
    title = 'Improvement of scores during trainig',
    xaxis = dict(title = 'Number of Episodes'),
    yaxis = dict(title = 'Score'),
)

fig = dict(data=data, layout=layout)
iplot(fig)